In [10]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import common.common_machine_learning as common
import common.feature_num as feature_num
import common.features_datasets_externos as feature

In [11]:
TARGET = "precio"
MAX_ITER = 10

### Funciones auxiliares

In [41]:
def limpiar(df):
    df.antiguedad = df.antiguedad.fillna(df.antiguedad.mean())
    df.metroscubiertos = df.metroscubiertos.fillna(df.metroscubiertos.mean())
    df.habitaciones = df.habitaciones.fillna(df.habitaciones.mean())
    df.garages = df.garages.fillna(0)
    df.banos = df.banos.fillna(1)
    df.tipodepropiedad = df.tipodepropiedad.fillna('Casa')
    df.metroscubiertos = df.metroscubiertos.fillna(df.metroscubiertos.mean())
    df.metrostotales = df.metrostotales.fillna(df.metrostotales.mean())
    df.gimnasio = df.gimnasio.fillna(0)
    df.usosmultiples = df.usosmultiples.fillna(0)
    df.piscina = df.piscina.fillna(0)
    df.escuelascercanas = df.escuelascercanas.fillna(0)
    df.centroscomercialescercanos = df.centroscomercialescercanos.fillna(0)
    df["metroscubiertos"] = df["metroscubiertos"].fillna(df["metroscubiertos"].mean())
    df.fillna(value = {"tipodepropiedad" : df["tipodepropiedad"].mode().to_string(),
                        "provincia" : df["provincia"].mode().to_string(),
                        "ciudad": df["ciudad"].mode().to_string()}, inplace = True)
    
def evaluar_rf(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Performance del modelo:')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

# Preparo el dataset

## Carga de datos

In [13]:
train = pd.read_csv('sets_de_datos/train.csv', index_col = 0)
test = pd.read_csv('sets_de_datos/test.csv', index_col = 0)

In [14]:
train["fecha"] = pd.to_datetime(train["fecha"], format="%Y-%m-%d %H:%M:%S", errors='coerce')
test["fecha"] = pd.to_datetime(test["fecha"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

## Limpio las columnas que voy a usar

In [15]:
train["provincia"].replace(["", np.nan], [train["provincia"].mode(), train["provincia"].mode()], inplace=True)
test["provincia"].replace(["", np.nan], [test["provincia"].mode(), test["provincia"].mode()], inplace=True)

In [16]:
limpiar(train)
limpiar(test)

In [17]:
test['tipodepropiedad'] = test['tipodepropiedad'].str.replace('0    Casa',"Casa")
train = train[train["tipodepropiedad"].isin(test["tipodepropiedad"].unique())]
train.dropna(subset=["tipodepropiedad"], inplace=True)

## Cargo features adicionales

In [18]:
inicio = time.time()
train = feature.agregar_feature_pbi(train)
fin = time.time()

In [19]:
print("Tardó: {} minutos en agregar las features pbi.".format((fin - inicio) / 60))

Tardó: 7.520368051528931 minutos en agregar las features pbi.


In [20]:
inicio = time.time()
test = feature.agregar_feature_pbi(test)
fin = time.time()

In [21]:
print("Tardó: {} minutos en agregar las features pbi.".format((fin - inicio) / 60))

Tardó: 1.9885783513387045 minutos en agregar las features pbi.


In [22]:
train = feature_num.completar_lat_lng_con_provincias_y_ciudades(train)
train = feature_num.completar_lat_lng_con_idzona_mean(train)
feature_num.completar_lat_lng_con_promedio_Mexico(train)

In [23]:
train.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'pbi_campo', 'pbi_mineria',
       'pbi_energia_agua_gas', 'pbi_construccion',
       'pbi_industrias_manufactureras', 'pbi_comercio'],
      dtype='object')

In [24]:
test = feature_num.completar_lat_lng_con_provincias_y_ciudades(test)
test = feature_num.completar_lat_lng_con_idzona_mean(test)
feature_num.completar_lat_lng_con_promedio_Mexico(test)

In [25]:
train.head()

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,piscina,escuelascercanas,centroscomercialescercanos,precio,pbi_campo,pbi_mineria,pbi_energia_agua_gas,pbi_construccion,pbi_industrias_manufactureras,pbi_comercio
0,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116114,2.0,1.0,2.0,...,0.0,0.0,0.0,2273000.0,994,120,13989,45092,172748,280309
1,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,2.0,...,0.0,1.0,1.0,3600000.0,1186,295,11744,64787,158272,370801
2,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,2.0,...,0.0,0.0,0.0,1200000.0,69867,2605,14657,96269,283192,143550
3,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.000000,2.0,1.0,1.0,...,0.0,1.0,1.0,650000.0,19304,2869,19258,103309,346779,264426
4,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,1.0,...,0.0,0.0,0.0,1150000.0,76106,3013,15621,105854,305314,171948


In [26]:
train['ratio_cubierto'] = train.apply(lambda x: x['metroscubiertos']/x['metrostotales'] if x['metrostotales'] else 1, axis = 1)
test['ratio_cubierto'] = test.apply(lambda x: x['metroscubiertos']/x['metrostotales'] if x['metrostotales'] else 1, axis = 1)

## Me quedo con algunas features

In [27]:
columnas_train = ["tipodepropiedad", "antiguedad", "lat", "lng", "gimnasio", "usosmultiples", "piscina", "pbi_campo",
                  "pbi_mineria", "pbi_energia_agua_gas", "pbi_construccion", "pbi_industrias_manufactureras",
                  "pbi_comercio", "ratio_cubierto", "precio"]
columnas_test = ["tipodepropiedad", "antiguedad", "lat", "lng", "gimnasio", "usosmultiples", "piscina", "pbi_campo",
                  "pbi_mineria", "pbi_energia_agua_gas", "pbi_construccion", "pbi_industrias_manufactureras",
                  "pbi_comercio", "ratio_cubierto"]

In [28]:
train = train[columnas_train]
test = test[columnas_test]

In [29]:
train_OHE  = pd.get_dummies(train)

In [30]:
test_OHE = pd.get_dummies(test)

In [31]:
test_OHE.shape

(60000, 35)

# Tuneo

### Divido el train

In [32]:
X = train_OHE.drop([TARGET], axis = 1).copy().values
y = list(train_OHE[TARGET].copy())

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 0)

### Grilla de parápametros

In [34]:
hiperparametros = {
    "n_estimators": [100, 200, 250 ,300, 350 ,400, 500],
    "max_depth": [10, 20, 30, 40, 50, 60],
    "max_features": [1, 5, 'auto', 'sqrt', 'log2'],
    "min_samples_leaf": [1, 2, 4],
    "min_samples_split": [2, 5, 10],
}

### Tuneo

In [35]:
rf = RandomForestRegressor()

inicio = time.time()


rf_random = RandomizedSearchCV(estimator = rf, param_distributions = hiperparametros, n_iter = MAX_ITER, cv = 3, verbose=2, random_state=42, n_jobs = -1)

fin = time.time()

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 37.7min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...s

In [36]:
print("El tuneo tardó: {} minutos.".format((fin - inicio) / 60))

El tuneo tardó: 1.6729036966959637e-05 minutos.


In [37]:
print("Mejores parámetros:")
print(rf_random.best_params_)

Mejores parámetros:
{'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 40}


# Entrenamiento

In [38]:
mejores_hiperparametros = rf_random.best_params_

In [39]:
mejor_rf = RandomForestRegressor(n_estimators = mejores_hiperparametros["n_estimators"],
                                 min_samples_split = mejores_hiperparametros["min_samples_split"],
                                 min_samples_leaf = mejores_hiperparametros["min_samples_leaf"],
                                 max_features = mejores_hiperparametros["max_features"],
                                 max_depth = mejores_hiperparametros["max_depth"])
mejor_rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [42]:
evaluar_rf(mejor_rf, X_test, y_test)

Performance del modelo:
Average Error: 1014794.7956 degrees.
Accuracy = 36.78%.


36.78090522141504